In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot
from math import sqrt
from pytz import timezone

In [58]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras import optimizers

In [3]:
import talib

In [4]:
# Fix the random seed to reproducibility
np.random.seed(7)

In [5]:
lag = 20

In [6]:
def get_X_data():
    import dovahkiin as dk
    dp = dk.DataParser()
    X = dp.get_data("cu")
    return X

In [7]:
X = get_X_data()
X = pd.DataFrame({"close": X["close"]})
y = pd.Series(talib.SMA(X["close"].values, lag), index=X.index)

In [8]:
X = X["2012":]
y = y["2012":]

In [9]:
# Use difference in X as a predicator
# X = X.diff()
X = X.fillna(0)
y = y.fillna(0)

In [10]:
def timeseries_to_supervised(raw_time_series, lag):
    p = {}
    for i in range(1, lag+1):
        p["{}".format(i)] = raw_time_series.shift(i).fillna(0)
    p["0"] = raw_time_series
    
    if type(raw_time_series) is pd.Series:
        supervised_data = pd.DataFrame(p)
        supervised_data = pd.Panel({"0": supervised_data})
        supervised_data = supervised_data.swapaxes(0, 1).swapaxes(1, 2)
    else:
        supervised_data = pd.Panel(p)
    return supervised_data

In [11]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [12]:
def create_supervised_X(raw_time_series, lag):
    supervised_X = timeseries_to_supervised(X, lag)
    if not type(raw_time_series) is pd.Series:
        swaped_supervised_X = supervised_X.swapaxes(0, 1)
    else:
        swaped_supervised_X = supervised_X
    return swaped_supervised_X

In [13]:
supervised_X = create_supervised_X(X, lag=lag)
supervised_X = supervised_X.fillna(0)
supervised_X_values = supervised_X.values

In [14]:
supervised_X.shape

(500310, 21, 1)

In [15]:
supervised_X.items[350218:][0]

Timestamp('2016-03-01 13:28:00+0800', tz='Asia/Shanghai')

In [16]:
X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.3)

In [17]:
X_train = X_train[218:]
y_train = y_train[218:]
print(len(X_train))

350000


In [18]:
X_test = X_test[92:]
y_test = y_test[92:]
print(len(X_test))

150000


In [19]:
def scale(X_train, X_test):
    # scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
    scaler = Normalizer()
    # scaler = RobustScaler()
    scaler = scaler.fit(X_train)
    
    if type(X_train) is pd.Series:
        train_scaled = pd.Series(scaler.transform(X_train), index=X_train.index)
        test_scaled = pd.Series(scaler.transform(X_test), index=X_test.index)
    else:
        train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
        test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    return scaler, train_scaled, test_scaled

In [20]:
def inverse_scale(scaler, X_train_scaled, X_test_scaled):
    X_train = scaler.inverse_transform(X_train_scaled)
    X_test = scaler.inverse_transform(X_test_scaled)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train_scaled.index, columns=X_train_scaled.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test_scaled.index, columns=X_test_scaled.columns)
    return scaler, X_train, X_test

In [21]:
time_series_step = lag
features = X_train.shape[1]
batch_size = 1000

In [22]:
scaler, X_train_scaled, X_test_scaled = scale(X_train, X_test)

In [23]:
X_train_supervised_scaled = timeseries_to_supervised(X_train_scaled, time_series_step)
X_train_supervised_scaled = X_train_supervised_scaled.swapaxes(0, 1)

X_test_supervised_scaled = timeseries_to_supervised(X_test_scaled, time_series_step)
X_test_supervised_scaled = X_test_supervised_scaled.swapaxes(0, 1)

In [24]:
yscaler = MinMaxScaler()
yscaler.fit(y_train)
y_train_scaled = yscaler.transform(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [25]:
def LSTM_Model():
    model = Sequential()
    model.add(
        LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
         return_sequences=True, 
         activation="relu"
        ))
    model.add(LSTM(32, activation="relu", stateful=True))
    model.add(Dense(128))
    model.add(Dense(1))
    
    return model

In [26]:
def FullyConnected_Model():
    model = Sequential()
    model.add(Dense(128, input_shape=(time_series_step+1,)))
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [27]:
adam = optimizers.adam(lr=1e-6)

In [28]:
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=1.)

In [29]:
model = LSTM_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

In [30]:
model = FullyConnected_Model()
model.compile(loss="mean_squared_error", optimizer=adam, metrics=[ 'mse'])

In [31]:
def trainFullyConnected_network():
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            X_train_supervised_scaled.values[:, :, 0],
            y_train,
            epochs=1000,
            batch_size=1000,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )

In [33]:
trainFullyConnected_network()

Train on 280000 samples, validate on 70000 samples
Epoch 1/1000
280000/280000 [==============================] - 1s - loss: 2514040850.7429 - mean_squared_error: 2514040850.7429 - val_loss: 1402506106.5143 - val_mean_squared_error: 1402506106.5143
Epoch 2/1000
280000/280000 [==============================] - 0s - loss: 2514008633.1429 - mean_squared_error: 2514008633.1429 - val_loss: 1402481287.3143 - val_mean_squared_error: 1402481287.3143
Epoch 3/1000
280000/280000 [==============================] - 0s - loss: 2513975216.9143 - mean_squared_error: 2513975216.9143 - val_loss: 1402456148.1143 - val_mean_squared_error: 1402456148.1143
Epoch 4/1000
280000/280000 [==============================] - 0s - loss: 2513941249.8286 - mean_squared_error: 2513941249.8286 - val_loss: 1402430436.5714 - val_mean_squared_error: 1402430436.5714
Epoch 5/1000
280000/280000 [==============================] - 0s - loss: 2513906559.0857 - mean_squared_error: 2513906559.0857 - val_loss: 1402404143.5429 - val_

KeyboardInterrupt: 

### Use Raw Data to train moving average

In [31]:
raw_X_train = X_train.copy()

In [32]:
for i in range(1, lag+1):
    raw_X_train["{}".format(i)] = X_train.shift(i).fillna(0)

In [33]:
std_scaler = StandardScaler()
y_std_scaler = StandardScaler()

In [34]:
std_scaler.fit(raw_X_train)
y_std_scaler.fit(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [35]:
raw_X_train_scaled = std_scaler.transform(raw_X_train)

In [36]:
raw_y_train_scaled = y_std_scaler.transform(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [37]:
len(raw_X_train_scaled) == len(raw_y_train_scaled)

True

In [38]:
sgd = optimizers.SGD(lr=1e-7, decay=1e-8, momentum=0.9, nesterov=True)
adam = optimizers.adam(lr=1e-6)

In [39]:
adadelta = optimizers.Adadelta(lr=1e-7)

In [40]:
model = FullyConnected_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

In [41]:
def trainFullyConnected_network():   
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=120,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
        
    return model

In [42]:
model = trainFullyConnected_network()

Train on 280000 samples, validate on 70000 samples
Epoch 1/120
280000/280000 [==============================] - 1s - loss: 0.7463 - mean_squared_error: 0.7463 - val_loss: 1.9695 - val_mean_squared_error: 1.9695
Epoch 2/120
280000/280000 [==============================] - 0s - loss: 0.5900 - mean_squared_error: 0.5900 - val_loss: 1.5663 - val_mean_squared_error: 1.5663
Epoch 3/120
280000/280000 [==============================] - 0s - loss: 0.4662 - mean_squared_error: 0.4662 - val_loss: 1.2466 - val_mean_squared_error: 1.2466
Epoch 4/120
280000/280000 [==============================] - 0s - loss: 0.3684 - mean_squared_error: 0.3684 - val_loss: 0.9931 - val_mean_squared_error: 0.9931
Epoch 5/120
280000/280000 [==============================] - 0s - loss: 0.2912 - mean_squared_error: 0.2912 - val_loss: 0.7919 - val_mean_squared_error: 0.7919
Epoch 6/120
280000/280000 [==============================] - 0s - loss: 0.2302 - mean_squared_error: 0.2302 - val_loss: 0.6321 - val_mean_squared_err

280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0015 - val_mean_squared_error: 0.0015
Epoch 52/120
280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 53/120
280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 54/120
280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 55/120
280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 56/120
280000/280000 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 57/120
280000/280000 [===================

280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 6.1016e-04 - val_mean_squared_error: 6.1016e-04
Epoch 101/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 6.0386e-04 - val_mean_squared_error: 6.0386e-04
Epoch 102/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.9765e-04 - val_mean_squared_error: 5.9765e-04
Epoch 103/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.9153e-04 - val_mean_squared_error: 5.9153e-04
Epoch 104/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.8551e-04 - val_mean_squared_error: 5.8551e-04
Epoch 105/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.7958e-04 - val_mean_squared_error: 5.795

In [43]:
raw_X_test = X_test.copy()
for i in range(1, lag+1):
    raw_X_test["{}".format(i)] = X_test.shift(i).fillna(0)

In [44]:
raw_X_test_scaled = std_scaler.transform(raw_X_test)

In [45]:
predicated_y_test_scaled = model.predict(raw_X_test_scaled)

In [53]:
predicated_y = predicated_y.reshape(150000,)

In [54]:
y_test.shape

(150000,)

In [48]:
predicated_y = y_std_scaler.inverse_transform(predicated_y_test_scaled)

In [55]:
y_diff = predicated_y - y_test

In [57]:
y_diff.abs().sum() / y_test.shape[0]

127.79531025035769

In [ ]:
y_test_scaled = y_std_scaler.transform(y_test)

In [ ]:
model.evaluate(raw_X_test_scaled, y_test_scaled)

In [ ]:
model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=20,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )

In [ ]:
model.evaluate(raw_X_test_scaled, y_test_scaled)

In [ ]:
y_test_predicated_scaled = model.predict(raw_X_test_scaled)

In [ ]:
y_test_predicated = y_std_scaler.inverse_transform(y_test_predicated_scaled)

In [ ]:
y_test_predicated

In [ ]:
len(y_test) == len(y_test_predicated)

### Use Raw Data to Train Moving Average LSTM

In [ ]:
raw_X_train = X_train.copy()
for i in range(1, lag+1):
    raw_X_train["{}".format(i)] = X_train.shift(i).fillna(0)

X_std_scaler = StandardScaler()
y_std_scaler = StandardScaler()

X_std_scaler.fit(raw_X_train)
y_std_scaler.fit(y_train)

raw_X_train_scaled = X_std_scaler.transform(raw_X_train)
raw_y_train_scaled = y_std_scaler.transform(y_train)

In [ ]:
raw_X_test = X_test.copy()
for i in range(1, lag+1):
    raw_X_test["{}".format(i)] = X_test.shift(i).fillna(0)

raw_X_test_scaled = X_std_scaler.transform(raw_X_test)
raw_y_test_scaled = y_std_scaler.transform(y_train)

In [ ]:
raw_X_train_scaled = raw_X_train_scaled.reshape(raw_X_train_scaled.shape[0], time_series_step+1, 1)
raw_X_test_scaled = raw_X_test_scaled.reshape(raw_X_test_scaled.shape[0], time_series_step+1, 1)

In [ ]:
def LSTM_Model(lstm_layers=None, dense_layers=None):
    model = Sequential()
    
    if lstm_layers:
        for i in range(lstm_layers):
            model.add(
                LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
                 return_sequences=True, 
                 activation="relu"))
        model.add(LSTM(32, activation="relu", stateful=True))
    else:
        model.add(
            LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
             return_sequences=True, 
             activation="relu"
            ))
        model.add(LSTM(32, activation="relu", stateful=True))
        
    if dense_layers:
        for i in range(dense_layers):
            model.add(Dense(128, activation="sigmoid"))
        model.add(Dense(1))
    else:
        model.add(Dense(128))
        model.add(Dense(1))
    
    return model

In [ ]:
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=1.)

In [ ]:
adam = optimizers.adam(lr=1e-6)

In [ ]:
def trainLSTM_network():   
    model = LSTM_Model(6, 3)
    model.compile(loss="mean_squared_error", optimizer=adam, metrics=[ 'mse'])
    
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=1,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
    return model

In [ ]:
model = trainLSTM_network()

In [ ]:
model.predict(raw_X_test_scaled)

In [ ]:
model.predict(raw_X_train_scaled, batch_size=batch_size)
y_predicated_scaled = model.predict(raw_X_test_scaled, batch_size=batch_size)

In [ ]:
y_predicatted = y_std_scaler.inverse_transform(y_predicated_scaled)

In [ ]:
y_predicatted

In [ ]:
y_predicated_scaled

In [ ]:
model.reset_states()

In [ ]:
y_predicated_scaled = model.predict(raw_X_test_scaled, batch_size=batch_size)

### Use Raw Data to Train Moving Average by ConvNet


In [ ]:
def ConvNet_Model():
    model = Sequential()
    model.add(Conv1D()
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [ ]:
def trainConv_network():   
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=120,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
        
    return model